In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
spark = SparkSession \
    .builder \
    .appName('Spark with SQL') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/03 05:07:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
user_log = spark.read.json('sparkify_log_small.json')
user_log.take(1)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')]

In [6]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



## Create a view and run SQL queries

In [7]:
# In order to use SQL with Spark you need to first create a temp view
user_log.createOrReplaceTempView('user_log_table')

In [8]:
spark.sql('''
    select
        *
    from user_log_table
    limit 2;
''').show()

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|   Lily Allen|Logged In|Elizabeth|     F|            7|   Chase|195.23873| free|Shreveport-Bossie...|   PUT|NextSong|1512718541284|     5027|      

In [9]:
spark.sql('''
    select 
        count(*)
    from user_log_table;
''').show()

+--------+
|count(1)|
+--------+
|   10000|
+--------+



In [10]:
# Collect displays data as a list of Row objects
spark.sql('''
    select
        userid, firstname, page, song
    from user_log_table
    where userid = '1046';
''').collect()

[Row(userid='1046', firstname='Kenneth', page='NextSong', song='Christmas Tears Will Fall'),
 Row(userid='1046', firstname='Kenneth', page='NextSong', song='Be Wary Of A Woman'),
 Row(userid='1046', firstname='Kenneth', page='NextSong', song='Public Enemy No.1'),
 Row(userid='1046', firstname='Kenneth', page='NextSong', song='Reign Of The Tyrants'),
 Row(userid='1046', firstname='Kenneth', page='NextSong', song='Father And Son'),
 Row(userid='1046', firstname='Kenneth', page='NextSong', song='No. 5'),
 Row(userid='1046', firstname='Kenneth', page='NextSong', song='Seventeen'),
 Row(userid='1046', firstname='Kenneth', page='Home', song=None),
 Row(userid='1046', firstname='Kenneth', page='NextSong', song='War on war'),
 Row(userid='1046', firstname='Kenneth', page='NextSong', song='Killermont Street'),
 Row(userid='1046', firstname='Kenneth', page='NextSong', song='Black & Blue'),
 Row(userid='1046', firstname='Kenneth', page='Logout', song=None),
 Row(userid='1046', firstname='Kenneth'

In [12]:
spark.sql('''
    select distinct
        page
    from user_log_table
    order by page;
''').show()

+----------------+
|            page|
+----------------+
|           About|
|       Downgrade|
|           Error|
|            Help|
|            Home|
|           Login|
|          Logout|
|        NextSong|
|   Save Settings|
|        Settings|
|Submit Downgrade|
|  Submit Upgrade|
|         Upgrade|
+----------------+



## User UDF (User Defined Functions)

In [13]:
# You always need to register a udf before using it in SQL context
spark.udf.register('get_hour', lambda x: int(datetime.datetime.fromtimestamp(x / 1000.0).hour))

<function __main__.<lambda>(x)>

In [15]:
spark.sql('''
    select
        *
        ,get_hour(ts) as hour
    from user_log_table
    limit 1;
''').collect()

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046', hour='22')]

In [16]:
songs_in_hour = spark.sql('''
    select
        get_hour(ts) as hour
        ,count(*) as plays_per_hour
    from user_log_table
    where page = 'NextSong'
    group by hour
    order by cast(hour as int) asc
''')

In [17]:
songs_in_hour.show()

+----+--------------+
|hour|plays_per_hour|
+----+--------------+
|   0|           456|
|   1|           454|
|   2|           382|
|   3|           302|
|   4|           352|
|   5|           276|
|   6|           348|
|   7|           358|
|   8|           375|
|   9|           249|
|  10|           216|
|  11|           228|
|  12|           251|
|  13|           339|
|  14|           462|
|  15|           479|
|  16|           484|
|  17|           430|
|  18|           362|
|  19|           295|
+----+--------------+
only showing top 20 rows



## Convert results into a Pandas dataframe

In [18]:
songs_in_hour_pd = songs_in_hour.toPandas()

In [19]:
print(songs_in_hour_pd)

   hour  plays_per_hour
0     0             456
1     1             454
2     2             382
3     3             302
4     4             352
5     5             276
6     6             348
7     7             358
8     8             375
9     9             249
10   10             216
11   11             228
12   12             251
13   13             339
14   14             462
15   15             479
16   16             484
17   17             430
18   18             362
19   19             295
20   20             257
21   21             248
22   22             369
23   23             375
